In [6]:
################################### setting path ###################################
import sys
sys.path.append('../')
sys.path.append('../../')
################################### import libs ###################################
# from  pytorchyolo import  models_split_large
from pytorchyolo import models
import numpy as np
import torch
from torchinfo import summary


In [7]:
cfg_path = "../ckpt/yolov3.cfg"
model_path = "../ckpt/stmarc_full.pth"

split_layer=21


In [8]:
model = models.load_model(cfg_path, model_path)
# model.set_split_layer(split_layer) # layer <7
model = model.eval()
summary(model, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 10647, 85]            --
├─ModuleList: 1-1                        --                        --
│    └─Sequential: 2-1                   [1, 32, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 32, 416, 416]         864
│    │    └─BatchNorm2d: 3-2             [1, 32, 416, 416]         64
│    │    └─LeakyReLU: 3-3               [1, 32, 416, 416]         --
│    └─Sequential: 2-2                   [1, 64, 208, 208]         --
│    │    └─Conv2d: 3-4                  [1, 64, 208, 208]         18,432
│    │    └─BatchNorm2d: 3-5             [1, 64, 208, 208]         128
│    │    └─LeakyReLU: 3-6               [1, 64, 208, 208]         --
│    └─Sequential: 2-3                   [1, 32, 208, 208]         --
│    │    └─Conv2d: 3-7                  [1, 32, 208, 208]         2,048
│    │    └─BatchNorm2d: 3-8             [1, 32, 208, 208]         64
│    │

In [4]:
input_tensor = torch.rand(1,3, 416,416).cuda()
head_output = model(input_tensor,1)
head_output.shape

0
1
2
3
4
4
-3
5
6
7
8
8
-3
9
10
11
11
-3
12
13
14
15
15
-3
16
17
18
18
-3
19
20
21
21
-3
22


torch.Size([1, 256, 52, 52])

In [5]:
output = model(head_output,2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2
-3


IndexError: list index out of range

In [10]:
output

tensor([[[ 15.6526,  17.5412,  56.1137,  ...,   0.4709,   0.4709,   0.4709],
         [ 48.0860,  17.6755,  56.8738,  ...,   0.4713,   0.4713,   0.4713],
         [ 79.9647,  18.0097,  57.0179,  ...,   0.4710,   0.4710,   0.4710],
         ...,
         [395.9532, 412.0807,  18.6175,  ...,   0.4923,   0.4923,   0.4923],
         [403.9572, 412.0856,  18.6066,  ...,   0.4917,   0.4917,   0.4917],
         [411.9361, 412.0955,  18.4215,  ...,   0.4866,   0.4866,   0.4866]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [23]:
# summary(model, input_size=(1, 3, 416, 416), dtypes=[torch.float16])

In [28]:
model_int8 = torch.quantization.quantize_dynamic(
    model,  # your model
    {torch.nn.Linear},  # layers to quantize
    dtype=torch.qint8
)

In [32]:
x = torch.randn(1, 3, 416, 416)
x_q = torch.quantize_per_tensor(x, scale=0.1, zero_point=128, dtype=torch.quint8)

summary(model_int8, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 256, 26, 26]          --
├─ModuleList: 1-1                        --                        8,459,262
│    └─Sequential: 2-1                   [1, 16, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 16, 416, 416]         432
│    │    └─BatchNorm2d: 3-2             [1, 16, 416, 416]         32
│    │    └─LeakyReLU: 3-3               [1, 16, 416, 416]         --
│    └─Sequential: 2-2                   [1, 16, 208, 208]         --
│    │    └─MaxPool2d: 3-4               [1, 16, 208, 208]         --
│    └─Sequential: 2-3                   [1, 32, 208, 208]         --
│    │    └─Conv2d: 3-5                  [1, 32, 208, 208]         4,608
│    │    └─BatchNorm2d: 3-6             [1, 32, 208, 208]         64
│    │    └─LeakyReLU: 3-7               [1, 32, 208, 208]         --
│    └─Sequential: 2-4                   [1, 32, 104, 104]         --
│   